# Advanced Programme in Deep Learning (Foundations and Applications)
## A Program by IISc and TalentSprint
### Assignment 7: Numerical Optimization

## Learning Objectives

At the end of the assignment, you will be able to

*   understand about constrained optimization

*   understand about Support Vector Machines

*   understand the use of Lagrange Multiplers with multiple constraints  

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (normally your phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython
import warnings
warnings.filterwarnings("ignore")

ipython = get_ipython()

notebook= "M1_AST_07_Numerical_Optimization_C" #name of the notebook

def setup():
    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer1() and getAnswer2() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer1" : Answer1, "answer2" : Answer2, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://dlfa-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer1():
  try:
    if not Answer1:
      raise NameError
    else:
      return Answer1
  except NameError:
    print ("Please answer Question 1")
    return None

def getAnswer2():
  try:
    if not Answer2:
      raise NameError
    else:
      return Answer2
  except NameError:
    print ("Please answer Question 2")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



### Importing required packages

In [ ]:
import numpy as np
from sklearn.svm import SVC                      # Importing Support vector classifier from sklearn
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.optimize import minimize, LinearConstraint           # Minimization of scalar function of one or more variables and Find the roots of a function
from scipy.linalg import solve                                          # Solves the linear equation set a * x = b for the unknown x for square a matrix

## Introduction to Optimization

Optimization is an important tool in decision science and the analysis of physical systems. To make use of this tool, first identify some objective, a quantitative measure of the performance of the system under study. This objective could be profit, time, potential energy, or any quantity or combination of quantities that can be represented by a single number. The objective depends on certain characteristics of the system, called variables or unknowns. Here, the goal is to find the values of the variables that optimize the objective function.

The process of identifying objective, variables, and constraints for a given problem is known as *modeling*. This step is mostly the first and the most important step in the optimization process. After an optimization algorithm has been applied to the model, we must be able to recognize whether it has succeeded in its task of finding a solution.



#### Constrained optimization

The general **constrained optimization** task is to maximize or minimize a function $f(x)$ by varying $x$, given certain constraints on $x$.

Eg. Find minimum of $f(x_1,x_2,x_3) = x_1^2 + x_2^2 + x_3^2,  where \lVert x_2\rVert \geq1$

Eg. Designing the fastest vehicle with a constraint on fuel efficiency

An optimization problem comprises of three basic components:

- $x$: is the vector of variables, also called unknowns or parameters;
- $f$: is the objective function, a (scalar) function of x that we want to maximize or
minimize;
- $c_i$: are constraint functions, which are scalar functions of x that define certain equations
and inequalities that the unknown vector x must satisfy.

Using this notation, the optimization problem can be written as follows:

$\min\limits_{x \in R^{N}}$ $f(x)$  

$\textrm{subject to}$ $\hspace{1cm} c_i(x) = 0, \hspace{0.25cm} i \in \xi$

$\hspace{2.75cm}$ $c_i(x)\geq0, \hspace{0.25cm} i \in I$

Here $I$ and $\xi$ are sets of indices for equality and inequality constraints.



#### Equality and Inequality Constraints

Consider the following general constrained optimization problem:

$\max\limits_{x_i \in R}$ $f(x_1,. . . . , x_n)$

subject to:

$g_1(x_1, . . . , x_n) \leq b_1, . . . , g_k(x_1, . . . , x_n) \leq b_k$,

$h_1(x_1, . . . , x_n) = c_1, . . . , h_m(x_1, . . . , x_n) = c_m$

The function $f(x)$ is called the objective function, $g(x)$ is called an *inequality constraint*, and $h(x)$ is called an *equality constraint*. In the above problem, there are $k$ *inequality constraints* and $m$ *equality constraints*. In the following, we will always assume that $f, g$, and h are $C^{1}$ functions, i.e. that they are differentiable and their derivatives are continuous.

Notice that this problem differs from the regular unconstrained optimization problem instead of finding the maximum of $f(x)$ we are finding the maximum of $f(x)$ only over the points which satisfy the constraints


### Example: Stockbroker Income

Let’s try a demonstration on how to use `minimize()`. Imagine you’re a stockbroker who’s interested in maximizing the total income from the sale of a fixed number of your stocks. You have identified a particular set of buyers, and for each buyer, you know the price they’ll pay and how much cash they have on hand.

There is one constraint on the problem, which is that the sum of the total shares purchased by the buyers does not exceed the number of shares you have on hand. There are also bounds on each of the solution variables because each buyer has an upper bound of cash available, and a lower bound of zero. Negative solution x-values mean that you’d be paying the buyers!

Steps we will follow to solve the problem:

1. Initialize the variables

2. Create arrays for storing price, money available with the buyers, and shares per buyer.

3. Setting the constraints

4. Setting the bounds

5. Declaring the 'objective function'.

6. Applying constrained optimization using minimize() method from scipy.

Solving Optimization Problem

In [ ]:
# set variables to determine the number of buyers in the market and the number of shares you want to sell
n_buyers = 10
n_shares = 15

Next, create arrays to store the price that each buyer pays for the shares, the maximum amount they can afford to spend, and the maximum number of shares each buyer can afford, given the first two arrays.

In [ ]:
np.random.seed(10)

# Generating the array of prices the buyers will pay
# np.random.random() creates an array of random numbers
# on the half-open interval [0,1)
prices = 100*np.random.random(n_buyers)

# generate an array of integers on the half-open interval from [1, 4),
# again with the size of the number of buyers
money_available = 100*np.random.randint(1, 4, n_buyers)

print("Price of the shares 1-{} : {}".format(n_shares, prices))
print("Money available with each buyer (1-{}): {}".format(n_buyers, money_available))

Now, you need to compute the maximum number of shares each buyer can afford:

In [ ]:
# take the ratio of the money_available with prices to determine the maximum number of shares each buyer can purchase
n_shares_per_buyer = money_available / prices

print("No of shares per buyer:\n",n_shares_per_buyer)

Now, you need to create the constraints and bounds for the solver.

 Remember that LinearConstraint takes the dot product of the input array with the solution values and compares it to the lower and upper bound. You can use this to set up the constraint on n_shares:

In [ ]:
# create an array of ones with the length `n_buyers` as `LinearConstraint` takes the first argument as matrix
# lb and ub are equal as it represents equality constraints
# For example, a machine component may be required to move precisely by Δ to perform the desired operation,
# so we must treat this as an equality constraint.
constraint = LinearConstraint(np.ones(n_buyers), lb = n_shares, ub = n_shares)

Since `LinearConstraint` takes the dot product of the solution vector with this argument, it’ll result in the sum of the purchased shares.

This result is then constrained to lie between the other two arguments:

1. The lower bound lb
2. The upper bound ub

Next, create the bounds for the solution variable. The bounds limit the number of shares purchased to be 0 on the lower side and `n_shares_per_buyer` on the upper side. The format that `minimize()` expects for the bounds is a sequence of tuples of lower and upper bounds:

In [ ]:
bounds = [(0, n) for n in n_shares_per_buyer]
bounds

In [ ]:
# Here the objective function returns negative of dot product of 'x' declared as 'x0' below
# and 'prices' which indicates that we are minimizing the value
def objective_function(x, prices):
    return -x.dot(prices)

In this code, you define `objective_function()` to take two arguments. Then you take the dot product of x with prices and return the negative of that value. Remember that you have to return the negative because you’re trying to make that number as small as possible, or as close to negative infinity as possible. Finally, you can call `minimize()`:

In [ ]:
res = minimize(
    objective_function,
    x0 = 10 * np.random.random(n_buyers),
    args = (prices),
    constraints=constraint,
    bounds=bounds,
)

In this code, res is an instance of OptimizeResult, just like with `minimize_scalar()`. As you’ll see, there are many of the same fields, even though the problem is quite different. In the call to `minimize()`, you pass five arguments:

1. objective_function
2. $x_0$
3. args
4. constraints
5. bounds

Once the solver runs, you should inspect `res` by printing it:

In [ ]:
res

You should also check and make sure that the constraints and bounds that you set are satisfied. You can do this with the following code:

In [ ]:
print("The total number of shares is:", sum(res.x))
print("Leftover money for each buyer:", money_available - res.x * prices)
print("Money Exhausted:\n",res.x * prices)

**Problem Statement**

Solve the problem of classification for linearly separable data, understand the concept of margin and get the essence of SVM (margin maximization).

### Introduction to Support Vector Machines

**Support vector machines** are a set of supervised learning methods used for classification, regression, and outliers detection.

A simple linear SVM classifier works by making a straight line between two classes. That means all of the data points on one side of the line will represent a category and the data points on the other side of the line will be put into a different category. This means there can be an infinite number of lines to choose from.

**Hyperplanes** are decision boundaries that help classify the data points. Data points falling on either side of the hyperplane can be attributed to different classes.
 * The hyperplane with maximum margin is called the optimal hyperplane.

**Support vectors** are data points that are closer to the hyperplane and influence the position and orientation of the hyperplane.

**Margin** is the width that the boundary could be increased by before hitting a data point.

![wget](https://cdn.talentsprint.com/aiml/aiml_2020_b14_hyd/experiment_details_backup/linear_data.png)







#### Support Vector Machine Algorithm

**Algorithm for Linearly separable, binary classification problem**:

Using the notation and steps provided by [Tristan Fletcher](https://static1.squarespace.com/static/58851af9ebbd1a30e98fb283/t/58902fbae4fcb5398aeb7505/1485844411772/SVM+Explained.pdf) the general steps to solve the SVM problem are the following:

1. Create P where $H_{i,j}= y^{(i)}.y^{(j)}<x^{(i)}.x^{(j)}>$

2. Calculate $w=\sum_{i}^{m}y^{(i)} \alpha_{i}x^{(i)}$

3. Determine the set of support vectors S by finding the indices such that $\alpha_{i}>0$

4. Calculate the intercept term using $ b = \frac{1}{N_{s}}\sum_{s\varepsilon  S} (y^{(s)}−\sum_{m ∈ S} \alpha_{m}y^{(m)}x^{(m)}.x^{(s)})$

5. Each new point $x'$ is classified by evaluating $y′=sgn(w^{T}x′+b)$

#### Mapping the problem according to the quadratic solvers API

Since we will solve this optimization problem using the [CVXOPT](https://cvxopt.org/userguide/coneprog.html#quadratic-programming) library in python we will need to match the solver's API which, according to the documentation is of the form:



 $min \frac{1}{2}x^{T}Px+q^{T}x$

subject to  

  * $Gx\leq h$ and,
  * $Ax=b$

With API

    cvxopt.solvers.qp(P, q[, G, h[, A, b[, solver[, initvals]]]])










The dual problem is expressed as:

$max \sum_{i=1}^{m} \alpha_{i}−\frac{1}{2}\sum_{i,j}^{m} y^{(i)}y^{(j)} \alpha_{i} \alpha_{j}<x^{(i)}x^{(j)}>$




Let $H$ be a matrix such that $H_{i,j}= y^{(i)}y^{(j)}<x^{(i)}x^{(j)}>$, then the optimization becomes:


$max \sum_{i=1}^{m} \alpha_{i}−\frac{1}{2} \alpha^{T} H \alpha$,  subject to constraints $\alpha_{i}\geq0$  and $\sum_{i=1}^{m}\alpha_{i} y^{(i)}=0$






We convert the sums into vector form and multiply both the objective and the constraint by −1 which turns this into a minimization problem and reverses the inequality

$min \frac{1}{2}\alpha^{T}H\alpha - 1^{T}\alpha$
subject to constraints $- \alpha_{i}\leq0$ and $y^{T}\alpha = 0$



We are now ready to convert our numpy arrays into the cvxopt format, using the same notation as in the documentation this gives

P:  $H$ a matrix of size $m×m$

q: $\vec{-1}$  a vector of size $m×1$

G: $−diag[1]$ a diagonal matrix of -1s of size $m×m$

h: $\vec{0}$  a vector of zeros of size $m×1$

A: $y$ the label vector of size $m×1$

b: $0$ a scalar

Note that in the simple example of $m=2$ the matrix $G$ and vector $h$ which define the constraint are

$$G = \begin{bmatrix}
-1 & 0 \\
0 & -1 \\
\end{bmatrix} and
\\
$$ $$h = \begin{bmatrix}
0 \\
0 \\
\end{bmatrix}$$

#### Computing the matrix $H$ in vectorized form


Consider the simple example with 2 input samples $(\left\{x^{(1)}, x^{(2)}\right\} \in \mathbb{R}^{2})$ which are two dimensional vectors. i.e. $
x^{(1)}=\left(x_{1}^{(1)}, x_{2}^{(1)}\right)^{T}$

$$X = \begin{bmatrix}
x^{(1)}_{1} & x^{(1)}_{2} \\
x^{(2)}_{1} & x^{(2)}_{2} \\
\end{bmatrix}, y = \begin{bmatrix}
y^{(1)}\\
y^{(2)}\\
\end{bmatrix}$$

We now proceed to creating a new matrix $X$ where each input sample $x$ is multiplied by the coresponding output label $y$.
This can be done easily in Numpy using vectorization and padding.

$$X' = \begin{bmatrix}
x^{(1)}_{1}y^{(1)} & x^{(1)}_{2}y^{(1)} \\
x^{(2)}_{1}y^{(2)} & x^{(2)}_{2}y^{(2)} \\
\end{bmatrix}$$

Finally we take the **matrix multiplication** of $X′$ and its transpose giving $H=X'X'T$


$$H = X'@X'^{T} = \begin{bmatrix}
x^{(1)}_{1}y^{(1)} & x^{(1)}_{2}y^{(1)} \\
x^{(2)}_{1}y^{(2)} & x^{(2)}_{2}y^{(2)} \\
\end{bmatrix}
\begin{bmatrix}
x^{(1)}_{1}y^{(1)} & x^{(2)}_{1}y^{(2)} \\
x^{(1)}_{2}y^{(1)} & x^{(2)}_{2}y^{(2)} \\
\end{bmatrix}$$


$$
H = \begin{bmatrix}
x_{1}^{(1)} x_{1}^{(1)} y^{(1)} y^{(1)}+x_{2}^{(1)} x_{2}^{(1)} y^{(1)} y^{(1)} & x_{1}^{(1)} x_{1}^{(2)} y^{(1)} y^{(2)}+x_{2}^{(1)} x_{2}^{(2)} y^{(1)} y^{(2)} \\
x_{1}^{(2)} x_{1}^{(1)} y^{(2)} y^{(1)}+x_{2}^{(2)} x_{2}^{(1)} y^{(2)} y^{(1)} & x_{1}^{(2)} x_{1}^{(2)} y^{(2)} y^{(2)}+x_{2}^{(2)} x_{2}^{(2)} y^{(2)} y^{(2)}
\end{bmatrix}
$$

#### Creating custom dataset

Let us create the custom data for our problem.

In [ ]:
# Creating data points
x_neg = np.array([[3,4],[1,4],[2,3]])
y_neg = np.array([-1,-1,-1])
x_pos = np.array([[6,-1],[7,-1],[5,-3]])
y_pos = np.array([1,1,1])
x1 = np.linspace(-10,10)
x = np.vstack((np.linspace(-10,10),np.linspace(-10,10)))

# Data for the next section
X = np.vstack((x_pos, x_neg))
y = np.concatenate((y_pos,y_neg))

In [ ]:
# Parameters (weight and bias) guessed by inspection
w = np.array([1,-1]).reshape(-1,1)
b = -3

#### Representing Support Vectors and Hyperplane

Let us plot the figure for representing the support vectors and drawing hyperplane to show whether the points are linearly separable or not.

In [ ]:
# Creating the plot
fig = plt.figure(figsize = (10,10))
plt.scatter(x_neg[:,0], x_neg[:,1], marker = 'x', color = 'r', label = 'Negative -1')
plt.scatter(x_pos[:,0], x_pos[:,1], marker = 'o', color = 'b',label = 'Positive +1')
plt.plot(x1, x1  - 3, color = 'darkblue')
plt.plot(x1, x1  - 7, linestyle = '--', alpha = .3, color = 'b')
plt.plot(x1, x1  + 1, linestyle = '--', alpha = .3, color = 'r')
plt.xlim(0,10)
plt.ylim(-5,5)
plt.xticks(np.arange(0, 10, step=1))
plt.yticks(np.arange(-5, 5, step=1))

# Adding the margin lines
plt.axvline(0, color = 'black', alpha = .5)
plt.axhline(0,color = 'black', alpha = .5)
plt.plot([2,6],[3,-1], linestyle = '-', color = 'darkblue', alpha = .5 )
plt.plot([4,6],[1,1],[6,6],[1,-1], linestyle = ':', color = 'darkblue', alpha = .5 )
plt.plot([0,1.5],[0,-1.5],[6,6],[1,-1], linestyle = ':', color = 'darkblue', alpha = .5 )

# Annotations
plt.annotate('$A \ (6,-1)$', xy = (5,-1), xytext = (6,-1.5))
plt.annotate('$B \ (2,3)$', xy = (2,3), xytext = (2,3.5)) #, arrowprops = {'width':.2, 'headwidth':8})
plt.annotate('$2$', xy = (5,1.2), xytext = (5,1.2) )
plt.annotate('$2$', xy = (6.2,.5), xytext = (6.2,.5))
plt.annotate('$2\sqrt{2}$', xy = (4.5,-.5), xytext = (4.5,-.5))
plt.annotate('$2\sqrt{2}$', xy = (2.5,1.5), xytext = (2.5,1.5))
plt.annotate('$w^Tx + b = 0$', xy = (8,4.5), xytext = (8,4.5))
plt.annotate('$(\\frac{1}{4},-\\frac{1}{4}) \\binom{x_1}{x_2}- \\frac{3}{4} = 0$', xy = (7.5,4), xytext = (7.5,4))
plt.annotate('$\\frac{3}{\sqrt{2}}$', xy = (.5,-1), xytext = (.5,-1))

# Labels and showing the plot
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend(loc = 'lower right')
plt.show()

#### Declaration of Parameters

Initializing values and computing $H$. Note that the 1. is used to convert y to float type.


In [ ]:
# Shape of the data points
m, n = X.shape

# Reshaping y column
y = y.reshape(-1,1) * 1.

# Input to the H matrix
X_dash = y * X
H = np.dot(X_dash , X_dash.T) * 1.

In [ ]:
# Converting into cvxopt format
P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(-np.eye(m))
h = cvxopt_matrix(np.zeros(m))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

Setting the solver parameters (change default to decrease tolerance)


In [ ]:
cvxopt_solvers.options['show_progress'] = False
cvxopt_solvers.options['abstol'] = 1e-10
cvxopt_solvers.options['reltol'] = 1e-10
cvxopt_solvers.options['feastol'] = 1e-10

#### Computation

Let us run the solver method in order to compute values of alpha, so that, it can be used further to compute w and b.

In [ ]:
# Run solver and find the values of alpha
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

Further, we will be computing the value w and b parameters.

In [ ]:
# w parameter in vectorized form
w = ((y * alphas).T @ X).reshape(-1,1)

# Selecting the set of indices S corresponding to non zero parameters
S = (alphas > 1e-4).flatten()

# Computing b
b = y[S] - np.dot(X[S], w)

# Display results
print('Alphas = ',alphas[alphas > 1e-4])
print('w = ', w.flatten())
print('b = ', b[0])

In [ ]:
# Selecting the alphas for support vectors
support_vector_boolean = (alphas > 1e-4).reshape(-1)

In [ ]:
print("Following are the support vectors:\n{}".format(X[support_vector_boolean]))

Draw the hyperplane with the updated parameters between the datapoints

In [ ]:
plt.scatter(x_neg[:, 0], x_neg[:,1])
plt.scatter(x_pos[:, 0], x_pos[:,1])
X1 = np.linspace(0, 10, 11)
X2 = (-b[0]-w[0]*X1)/w[1]
plt.plot(X1, X2);

In [ ]:
print(len(X2))

Now, let us compare it to the Sklearn results




#### Comparing the outputs with the sklearn results

**What is scikit learn?**

* Scikit learn is a library used to perform machine learning in Python. Scikit learn is an open source machine learning library which is free software licensed and is reusable in various contexts, encouraging academic and commercial use.

* Scikit-learn provides a range of supervised and unsupervised learning algorithms in Python.

* It provides efficient tools for data analysis, data pre-processing, model building, model evaluation, and much more.

Refer to the follwing [link](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) for the SVC classifier from sklearn

In [ ]:
# Declaring the Support Vector Classifier
clf = SVC(kernel = 'linear')

* First we start with importing the necessary classifier and creating an instance for it.

* By using **fit()** we fit the classifier to the training points, which  is essentially the training part of the modeling process

In [ ]:
%%time
clf.fit(X, y.ravel())

In [ ]:
a = [1, 2, 3, 4, 5]
b = [6, 7, 8, 9, 10]
c = list(map(lambda x: sum(x), zip(a, b)))
print(c)

In [ ]:
print('w (coefficient) = ',clf.coef_)
print('b (Intercept) = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

### Constrained Optimization

Constrained optimization problems are problems for which a function is to be minimized or maximized subject to constraints. Our constrained optimization problem is to maximize the function $f(x, y)$ while satisfying the constraint $g(x, y) = 0$ . maximize $f(x, y)$ subject to $g(x, y) = 0$. In some other scenarios, an optimization could be a minimization problem. minimize $f(x, y)$ subject to $g(x, y) = 0$

In [ ]:
import numpy as np
from scipy.linalg import solve

#### Lagrange multipliers and constrained optimization

Recall why Lagrange multipliers are useful for constrained optimization - a stationary point must be where the constraint surface $g$ touches a level set of the function $f$ (since the value of $f$ does not change on a level set). At that point, $f$ and $g$ are parallel, and hence their gradients are also parallel (since the gradient is normal to the level set). So we want to solve

<center>

$ \bigtriangledown f  = - λ \bigtriangledown g$

</center>

or equivalently,

<center>
$ \bigtriangledown f + λ \bigtriangledown g = 0 $
</center>

<br><br>
<center>

![image.png](https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/LagrangeMultipliers2D.svg/300px-LagrangeMultipliers2D.svg.png)

Lagrange multipliers


</center>



1. Maximize the function $f(x,y,z) = xy + yz$  subject to the two constraints: $x + 2y = 6$ and $x − 3z = 0$.

We set up the equations

<center>

$F(x, y, z, λ, μ) = xy + yz − λ(x + 2y − 6) − μ(x − 3z)$

<br>

</center>

Now set partial derivatives to zero and solve the following set of equations

<center>



$\frac {∂F}{∂x} =  y − λ − μ = 0 $ \\

$\frac {∂F}{∂y} =  x + z − 2λ = 0 $ \\

$\frac {∂F}{∂z} =  y + 3μ = 0$ \\

$\frac {∂F}{∂λ} =  x + 2y − 6 = 0 $ \\

$\frac {∂F}{∂μ} =  x − 3z = 0 $ \\

</center>

<br>

Plugging in the numbers and expressing in matrix form, we get which is a linear equation in $x, y, z, λ, μ$

<center>
<br>

\begin{equation}
  \begin{bmatrix}
    0 & 1 & 0 & -1 & -1\\
1 & 0 & 1 & -2 & 0\\
0 & 1 & 0 & 0 & 3\\
1 & 2 & 0 & 0 & 0\\
1 & 0 & -3 & 0 & 0
  \end{bmatrix}
  %
  \begin{bmatrix}
    x \\
    y \\
    z \\
    λ \\
    μ
  \end{bmatrix}
  %
  =
   \begin{bmatrix}
    0 \\
    0 \\
    0 \\
    6 \\
    0 \\
  \end{bmatrix}
\end{equation}




In [ ]:
# Defining the above obtained matrix
A = np.array([
    [0, 1, 0, -1, -1],
    [1, 0, 1, -2, 0],
    [0, 1, 0, 0, 3],
    [1, 2, 0, 0, 0],
    [1, 0,-3, 0, 0]])

b = np.array([0,0,0,6,0])

sol = solve(A, b)

In [ ]:
print(sol)

In [ ]:
# Defining the original function
def f(x, y, z):
    return x * y + y * z

In [ ]:
print(f(*sol[:3]))

### Please answer the questions below to complete the experiment:




In [ ]:
#@title Q.1. Use Lagrange multipliers to find the maxima and minima values of f(x, y) = 3x−4y subject to the constraint x^2+3y^2 = 129, if such values exist. { run: "auto", form-width: "500px", display-mode: "form" }
Answer1 = "Maxima = 11, Minima = -11" #@param ["","Maxima = 11, Minima = -11", "Maxima = 48, Minima = -48", "Maxima = 43, Minima = -43", "Maxima = 24, Minima = -24"]


In [ ]:
#@title Q.2. In Support Vector Machines, the problem of finding a separating hyperplane with the maximum margin (the optimal hyper plane) is a constrained optimization problem?  { run: "auto", form-width: "500px", display-mode: "form" }
Answer2 = "True" #@param ["","True", "False"]


In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")